In [1]:
import cobra
import re
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
from metnet import pathway 
import sys
sys.path.append(r'./')
from COBRApy_ECM_funcation import *
from metnet import sbml_excel
import os
from copy import copy, deepcopy
import numpy as np
from cobra.io import write_sbml_model,save_json_model
from ec_code.cobrapy_ec_model_function import convert_to_irreversible,isoenzyme_split

In [2]:
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model

#### model json file

In [3]:
model_1515_773_file='./iCW773+iML1515/model/iML1515_iCW773.json'

In [847]:
model_origin_1515='./iCW773+iML1515/model/iML1515_new.json'
model_origin_773='./iCW773+iML1515/model/iCW773_final.json'

In [7]:
model1=load_json_model(model_origin_1515)
model1.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model1.reactions.get_by_id('EX_xyl__D_e').bounds=(-1000,1000)
# model.reactions.get_by_id('EX_lcts_e').bounds=(-2,1000)
# model.reactions.get_by_id('EX_fru_e').bounds=(-10,1000)
model1.optimize()
model1.summary()

IN FLUXES            OUT FLUXES       OBJECTIVES
-------------------  ---------------  ---------------------
xyl__D_e  757        for_e     1e+03  BIOMASS_Ec_i...  25.4
o2_e      623        h2o_e     1e+03
fe2_e     609        h_e       1e+03
nh4_e     274        fe3_e   609
pi_e       24.5      ac_e    376
so4_e       6.39     co2_e   348
k_e         4.95     etoh_e  323
mg2_e       0.22
ca2_e       0.132
cl_e        0.132
cu2_e       0.018
mn2_e       0.0175
zn2_e       0.00866
ni2_e       0.0082


In [8]:
model1.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_xyl__D_e': 1000,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [9]:
model2=load_json_model(model_origin_773)
model2.reactions.get_by_id('EX_glc_e').bounds=(-1000,1000)
model2.reactions.get_by_id('EX_xyl__D_e').bounds=(-1000,1000)
model2.optimize()
model2.summary()

IN FLUXES            OUT FLUXES          OBJECTIVES
-------------------  ------------------  ---------------------
glc__D_e   564       h2o_e      1e+03    CG_biomass_c...  25.1
o2_e       381       co2_e    981
nh4_e      209       etoh_e   518
k_e         31.8     h_e      341
pi_e        12.5     ac_e     155
so4_e        4.02    thymd_e    4.03
mg2_e        1.41    glyc_e     0.00477
fe3_e        1.27
fe2_e        1.09
ca2_e        0.847
cl_e         0.847
cu2_e        0.115
mn2_e        0.113
zn2_e        0.0555
ni2_e        0.0525
cobalt2_e    0.0041


In [10]:
model2.medium

{'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc_e': 1000,
 'EX_xyl__D_e': 1000,
 'EX_ca2_e_reverse': 1000.0,
 'EX_co2_e_reverse': 1000.0,
 'EX_cobalt2_e_reverse': 1000.0,
 'EX_cu2_e_reverse': 1000.0,
 'EX_glc_e_reverse': 4.67,
 'EX_h2o_e_reverse': 1000.0,
 'EX_k_e_reverse': 1000.0,
 'EX_mg2_e_reverse': 1000.0,
 'EX_mn2_e_reverse': 1000.0,
 'EX_na1_e_reverse': 1000.0,
 'EX_nh4_e_reverse': 1000.0,
 'EX_ni2_e_reverse': 1000.0,
 'EX_o2_e_reverse': 1000.0,
 'EX_pi_e_reverse': 1000.0,
 'EX_so4_e_reverse': 1000.0,
 'EX_zn2_e_reverse': 1000.0,
 'EX_BIOTIN_reverse': 1000.0,
 'EX_cl_reverse': 1000.0}

iBsu773 information file

In [11]:
def sbml_to_excel(modelname,output):#模型SBML文件转为excel文件
    from cobra.io.dict import model_to_dict
    from cobra.io import read_sbml_model
    import pandas as pd
    model=read_sbml_model(modelname)
    model_dict=model_to_dict(model,sort=False)
    writer = pd.ExcelWriter(output)
    pd.DataFrame(model_dict['metabolites']).to_excel(writer,'metabolites',index=False)
    pd.DataFrame(model_dict['genes']).to_excel(writer,'genes',index=False)
    model_df=pd.DataFrame(model_dict['reactions'])
    model_df['reaction_eq'] = model_df.id.apply(lambda x: model.reactions.get_by_id(x).build_reaction_string(use_metabolite_names=False))
    model_df['reaction_eq_name'] = model_df.id.apply(lambda x: model.reactions.get_by_id(x).build_reaction_string(use_metabolite_names=True))
    model_df.to_excel(writer,'reactions',index=False)
    writer.save()

In [12]:
def json_to_excel(modelname,output):#模型json文件转为excel文件
    from cobra.io.dict import model_to_dict
    from cobra.io import load_json_model
    import pandas as pd
    model=load_json_model(modelname)
    model_dict=model_to_dict(model,sort=False)
    writer = pd.ExcelWriter(output)
    pd.DataFrame(model_dict['metabolites']).to_excel(writer,'metabolites',index=False)
    pd.DataFrame(model_dict['genes']).to_excel(writer,'genes',index=False)
    model_df=pd.DataFrame(model_dict['reactions'])
    model_df['reaction_eq'] = model_df.id.apply(lambda x: model.reactions.get_by_id(x).build_reaction_string(use_metabolite_names=False))
    model_df['reaction_eq_name'] = model_df.id.apply(lambda x: model.reactions.get_by_id(x).build_reaction_string(use_metabolite_names=True))
    model_df.to_excel(writer,'reactions',index=False)
    writer.save()

In [ ]:
json_to_excel('./iCW773+iML1515/model/iCW773_final.json','./iCW773+iML1515/model/iCW773_final.xlsx')

In [ ]:
json_to_excel('./iCW773+iML1515/model/iML1515_new.json','./iCW773+iML1515/model/iMl1515_new.xlsx')

#### model construction

In [292]:
model1=load_json_model(model_origin_1515)

In [300]:
model1.metabolites.get_by_id('pi_c')

Metabolite identifier,pi_c
Name,Phosphate
Memory address,0x01b19d37bb70
Formula,HO4P
Compartment,c
In 324 reaction(s),"PPS, PRASCSi, PE140abcpp, PItpp, PGPP160, ACPPAT180, SELabcpp, PE141abcpp, LIPAabcpp, RIBabcpp, GTPDPDP, GNP, MALTabcpp, CYSabc2pp, GLUCYS, PGPP161, ACPPAT160, ACCOAC, PGP141abcpp, GLCP, GLUabcpp, ..."


reaction information

In [323]:
model1=load_json_model(model_origin_1515)
pydx5p_c=model1.metabolites.get_by_id('pydx5p_c')
lys__L_c=model1.metabolites.get_by_id('lys__L_c')
dap_c=model1.metabolites.get_by_id('15dap_c')
r5p_c=model1.metabolites.get_by_id('r5p_c')
g3p_c=model1.metabolites.get_by_id('g3p_c')
glu__L_c=model1.metabolites.get_by_id('glu__L_c')
gln__L_c=model1.metabolites.get_by_id('gln__L_c')
pi_c=model1.metabolites.get_by_id('pi_c')
h_c=model1.metabolites.get_by_id('h_c')
h2o_c=model1.metabolites.get_by_id('h2o_c')
EX_lys__L_e=model1.reactions.get_by_id('EX_lys__L_e')
EX_lys__L_e.bounds=(-1000,1000)
LYSDC=model1.reactions.get_by_id('LYSDC')
pdx=Reaction("pdx")
pdx.name='BioID:RXN-11322'
pdx.subsystem='unknow'
pdx.lower_bound=-1000
pdx.upper_bound=1000
pdx.add_metabolites({
    gln__L_c:-1.0,
    g3p_c:-1.0,
    r5p_c:-1.0,
    pydx5p_c:1.0,
    pi_c:1.0,
    glu__L_c:1.0,
    h2o_c:3.0,
    h_c:1.0
})
model1.add_reaction(pdx)
model1.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
#model1.reactions.get_by_id('EX_glc__D_e_reverse').bounds=(0,0)
model1.reactions.get_by_id('EX_glyc_e').bounds=(-1000,1000)
model_origin_1515_final=model_origin_1515.split('.json')[0]+'_final.json'
save_json_model(model1,model_origin_1515_final)

In [344]:
LYSDC

Reaction identifier,LYSDC
Name,Lysine decarboxylase
Memory address,0x01b1aa2a2860
Stoichiometry,"h_c + lys__L_c --> 15dap_c + co2_c H+ + L-Lysine --> 1,5-Diaminopentane + CO2 CO2"
GPR,b0186 or b4131
Lower bound,0.0
Upper bound,1000.0


In [324]:
model1.optimize()
model1.summary()

IN FLUXES            OUT FLUXES        OBJECTIVES
-------------------  ----------------  ---------------------
glyc_e    841        h2o_e      1e+03  BIOMASS_Ec_i...  22.7
lys__L_e  686        h_e        1e+03
o2_e      454        for_e    998
nh4_e     230        15dap_e  678
pi_e       21.9      ac_e     480
so4_e       5.72     co2_e    288
k_e         4.43     etoh_e    34.8
fe2_e       0.365
mg2_e       0.197
ca2_e       0.118
cl_e        0.118
cu2_e       0.0161
mn2_e       0.0157
zn2_e       0.00775
ni2_e       0.00734


In [303]:
model2=load_json_model(model_origin_773)
glyc_c=model2.metabolites.get_by_id('glyc_c')
dha_c=model2.metabolites.get_by_id('dha_c')
dhap_c=model2.metabolites.get_by_id('dhap_c')
#ASPK reaction up
lys__L_c=model2.metabolites.get_by_id('lys__L_c')
EX_lys_L_e=model2.reactions.get_by_id('EX_lys_L_e')
EX_glyc_e=model2.reactions.get_by_id('EX_glyc_e')
# metabolites
nad_c=model2.metabolites.get_by_id('nad_c')
nadh_c=model2.metabolites.get_by_id('nadh_c')
h_c=model2.metabolites.get_by_id('h_c')
DhaD=Reaction("DhaD")
DhaD.name='BioID:GLYCDEH-RXN'
DhaD.subsystem='unknow'
DhaD.lower_bound=-1000
DhaD.upper_bound=1000
DhaD.add_metabolites({
    glyc_c:-1.0,
    nad_c:-1.0,
    dha_c:1.0,
    nadh_c:1.0,
    h_c:1.0
})
# metabolites
atp_c=model2.metabolites.get_by_id('atp_c')
adp_c=model2.metabolites.get_by_id('adp_c')
DhaK=Reaction("DhaK")
DhaK.name='BioID:GLYCERONE-KINASE-RXN'
DhaK.subsystem='unknow'
DhaK.lower_bound=0
DhaK.upper_bound=1000
DhaK.add_metabolites({
    dha_c:-1.0,
    atp_c:-1.0,
    dhap_c:1.0,
    adp_c:1.0,
    h_c:1.0,
})
model2.add_reactions([DhaD,DhaK])
EX_glyc_e.bounds=(-1000,1000)
EX_lys_L_e.bounds=(-1000,1000)
model2.reactions.get_by_id('EX_glc_e').bounds=(0,1000)
model2.reactions.get_by_id('EX_glc_e_reverse').bounds=(0,0)
model_origin_773_lys=model_origin_773.split('.json')[0]+'_lys.json'
save_json_model(model2,model_origin_773_lys)

In [304]:
model2.optimize()
model2.summary()

IN FLUXES             OUT FLUXES         OBJECTIVES
--------------------  -----------------  ---------------------
glyc_e     482        h2o_e       1e+03  CG_biomass_c...  19.3
o2_e       298        h_e       312
nh4_e      150        lac__L_e  182
k_e         24.4      co2_e     126
pi_e         9.58     met__L_e    3.09
so4_e        6.18
lys__L_e     3.95
mg2_e        1.08
fe3_e        0.976
fe2_e        0.84
ca2_e        0.651
cl_e         0.651
cu2_e        0.0886
mn2_e        0.0865
zn2_e        0.0426
ni2_e        0.0404
cobalt2_e    0.00315


medium analyze

In [52]:
model1=load_json_model(model_origin_1515_final)
model1.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glyc_e': 1000,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_lys__L_e': 1000,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [54]:
model2=load_json_model(model_origin_773_lys)
model2.medium

{'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glyc_e': 1000,
 'EX_lys_L_e': 1000,
 'EX_ca2_e_reverse': 1000.0,
 'EX_co2_e_reverse': 1000.0,
 'EX_cobalt2_e_reverse': 1000.0,
 'EX_cu2_e_reverse': 1000.0,
 'EX_h2o_e_reverse': 1000.0,
 'EX_k_e_reverse': 1000.0,
 'EX_mg2_e_reverse': 1000.0,
 'EX_mn2_e_reverse': 1000.0,
 'EX_na1_e_reverse': 1000.0,
 'EX_nh4_e_reverse': 1000.0,
 'EX_ni2_e_reverse': 1000.0,
 'EX_o2_e_reverse': 1000.0,
 'EX_pi_e_reverse': 1000.0,
 'EX_so4_e_reverse': 1000.0,
 'EX_zn2_e_reverse': 1000.0,
 'EX_BIOTIN_reverse': 1000.0,
 'EX_cl_reverse': 1000.0}

In [57]:
for i in model2.medium:
    r=model2.reactions.get_by_id(i)
    #print(r.metabolites.keys())
    for j in r.metabolites.keys():
        print(j.id)

fe2_e
fe3_e
glyc_e
lys__L_e
ca2_e
co2_e
cobalt2_e
cu2_e
h2o_e
k_e
mg2_e
mn2_e
na1_e
nh4_e
ni2_e
o2_e
pi_e
so4_e
zn2_e
btn_e
cl_e


construct double-model

In [325]:
model_a=load_json_model(model_origin_1515_final)
model_b=load_json_model(model_origin_773_lys)
# model_a.reactions.get_by_id('EX_glc__D_e').bounds=(-1000,1000)
# model_a.reactions.get_by_id('EX_'+meta_test+'_e').bounds=(0,1000)
# model_b.reactions.get_by_id('EX_'+meta_test+'_e').bounds=(-1000,0)
model_a.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model_b.reactions.get_by_id('EX_glc_e').bounds=(0,1000)
model_a.reactions.get_by_id('EX_glyc_e').bounds=(-1000,1000)
model_b.reactions.get_by_id('EX_glyc_e').bounds=(-1000,1000)
model_b.reactions.get_by_id('EX_lys_L_e').bounds=(-1000,1000)
model_a.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
model_b.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(0.1,1000)
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name
for i in model_t.reactions:
    model.add_reaction(i)
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])
origin_model1=cobra.io.load_json_model(model_origin_1515_final)
origin_model1.medium
for i in origin_model1.medium:
    model.add_reaction(origin_model1.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    #model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
origin_model2=cobra.io.load_json_model(model_origin_773_lys)
origin_model2.medium
for i in origin_model2.medium:
    r=model2.reactions.get_by_id(i)
    #print(r.metabolites.keys())
    for j in r.metabolites.keys():
        ex_meta=j.id#print(j)
    if 'EX_'+ex_meta not in model.medium:
        model.add_reaction(origin_model2.reactions.get_by_id(i))
    #model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
# model.reactions.get_by_id('A_'+meta_test+'_e'+'_con_1').bounds=(-1000,1000)
# model.reactions.get_by_id('B_'+meta_test+'_e'+'_con_2').bounds=(-1000,1000)
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium
#model.medium
EX_15dap_e=model1.reactions.get_by_id('EX_15dap_e')
model.add_reaction(EX_15dap_e)
save_json_model(model,model_1515_773_file)

--------A  model----------
2714
1877
1516
--------B  model----------
1852
949
794
A_pi_e_con_1
A_h_e_con_1
A_fe3_e_con_1
A_glyc_e_con_1
A_mn2_e_con_1
A_co2_e_con_1
A_fe2_e_con_1
A_lys__L_e_con_1
A_zn2_e_con_1
A_mg2_e_con_1
A_ca2_e_con_1
A_ni2_e_con_1
A_cu2_e_con_1
A_cobalt2_e_con_1
A_sel_e_con_1
A_h2o_e_con_1
A_nh4_e_con_1
A_mobd_e_con_1
A_so4_e_con_1
A_k_e_con_1
A_na1_e_con_1
A_o2_e_con_1
A_cl_e_con_1
A_tungs_e_con_1
A_slnt_e_con_1
A_fe2_e_con_1
A_fe3_e_con_1
A_glyc_e_con_1
A_lys__L_e_con_1
A_ca2_e_con_1
A_co2_e_con_1
A_cobalt2_e_con_1
A_cu2_e_con_1
A_h2o_e_con_1
A_k_e_con_1
A_mg2_e_con_1
A_mn2_e_con_1
A_na1_e_con_1
A_nh4_e_con_1
A_ni2_e_con_1
A_o2_e_con_1
A_pi_e_con_1
A_so4_e_con_1
A_zn2_e_con_1
A_btn_e_con_1
A_cl_e_con_1


In [306]:
model.reactions.get_by_id('B_glyc_e_con_2')

Reaction identifier,B_glyc_e_con_2
Name,glyc_e Connection 2
Memory address,0x01b19e20b828
Stoichiometry,glyc_e <=> B_glyc_e Glycerol <=> B_Glycerol
GPR,
Lower bound,-1000
Upper bound,1000


In [76]:
model.metabolites.get_by_id('zn2_e')

Metabolite identifier,zn2_e
Name,Zinc
Memory address,0x01b1a3d9ea20
Formula,Zn
Compartment,e
In 3 reaction(s),"EX_zn2_e, B_zn2_e_con_2, A_zn2_e_con_1"


In [89]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glyc_e': 1000,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_lys__L_e': 1000,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_BIOTIN_reverse': 1000.0}

In [307]:
model.objective=model.reactions.get_by_id('EX_15dap_e')
model.optimize()
model.summary()

IN FLUXES            OUT FLUXES           OBJECTIVES
-------------------  -------------------  -----------------
lys__L_e    1e+03    15dap_e       1e+03  EX_15dap_e  1e+03
glyc_e    522        B_h2o_e     593
o2_e      258        co2_e       517
nh4_e      31.9      A_mal__L_e  475
pi_e        2.82     A_for_e      22.4
so4_e       0.74     A_ac_e        1.67
k_e         0.688    B_co2_e       1.24
fe2_e       0.0556   B_h_e         0.74
mg2_e       0.0306   B_his__L_e    0.016
ca2_e       0.0183
cl_e        0.0183
cu2_e       0.0025
mn2_e       0.00244
zn2_e       0.0012
ni2_e       0.00114


In [147]:
model=load_json_model(model_1515_773_file)
model

Name,iML1515
Memory address,0x01b1b7e15860
Number of metabolites,3183
Number of reactions,5145
Objective expression,1.0*A_BIOMASS_Ec_iML1515_core_75p37M - 1.0*A_BIOMASS_Ec_iML1515_core_75p37M_reverse_6059f
Compartments,"cytosol, extracellular space, periplasm"


In [149]:
model1.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glyc_e': 1000,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_lys__L_e': 1000,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [150]:
for i in model.medium:
    if i not in model1.medium:
        print(i)

EX_BIOTIN_reverse


In [152]:
model2.reactions.EX_BIOTIN

Reaction identifier,EX_BIOTIN
Name,EX_BIOTIN
Memory address,0x01b19f4d0320
Stoichiometry,btn_e --> Biotin -->
GPR,
Lower bound,0
Upper bound,1000.0


In [ ]:
model.reactions.get_by_id('A_fru_e_con_1')

In [ ]:
model.reactions.get_by_id('EX_fru_e')

In [ ]:
model.reactions.get_by_id('B_lys_e_con_2')

In [ ]:
model.reactions.get_by_id('A_lcts_e_con_1').bounds=(-1000,1000)
model.reactions.get_by_id('B_lcts_e_con_2').bounds=(-1000,1000)
model.reactions.get_by_id('A_fru_e_con_1').bounds=(-1000,1000)
model.reactions.get_by_id('B_fru_e_con_2').bounds=(-1000,1000)
save_json_model(model,model_1515_773_file)

In [ ]:
model=load_json_model(model_1515_773_file)
model.objective=model.reactions.get_by_id('EX_lys_e')
model.reactions.get_by_id('EX_glc__D_e').bounds=(0,1000)
model.reactions.get_by_id('EX_lcts_e').bounds=(-2,1000)
model.reactions.get_by_id('EX_fru_e').bounds=(-10,1000)
#model.reactions.get_by_id('EX_glc__D_e').bounds=(-100,1000)
model.optimize()
model.summary()

In [ ]:
model.reactions.get_by_id('EX_fru_e')

## main

In [4]:
from CulECpyD import *

In [506]:
model_file=model_1515_773_file
reaction_kcat_MW_file1='./iCW773+iML1515/data/iML1515_reaction_kcat_MW.csv'
reaction_kcat_MW_file2='./iCW773+iML1515/data/iCW773_reaction_kcat_MW.csv'
reaction_kcat_MW_file3='./iCW773+iML1515/data/iML1515_iCW773_reaction_kcat_MW.csv'
Concretemodel_Need_Data=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file1,'None')

In [507]:
def unpdate_kcat_mw_iBsu_iML(reaction_list,kcat_file_A,kcat_file_B,new_file):
    reaction_up_kact_MW=pd.DataFrame()
    kcat_A=pd.read_csv(kcat_file_A,index_col=0)
    kcat_B=pd.read_csv(kcat_file_B,index_col=0)
    for i in reaction_list:
        if i.startswith('A_'):
            id=i[2:]
            if id in kcat_A.index:
                reaction_up_kact_MW.at[i,'kcat_MW']=kcat_A.at[id,'kcat_MW']
        if i.startswith('B_'):
            id=i[2:]
            if id in kcat_B.index:
                reaction_up_kact_MW.at[i,'kcat_MW']=kcat_B.at[id,'kcat_MW'] 
    reaction_up_kact_MW.to_csv(new_file)
    return reaction_up_kact_MW


In [734]:
Concretemodel_Need_Data['reaction_kcat_MW']=unpdate_kcat_mw_iBsu_iML(Concretemodel_Need_Data['reaction_list'],reaction_kcat_MW_file1,reaction_kcat_MW_file2,reaction_kcat_MW_file3)

function

In [733]:
def Template_Concretemodel_double_diff(reaction_list=None,metabolite_list=None,coef_matrix=None,reaction_kcat_MW=None,lb_list=None,ub_list=None,reaction_list_A=None,reaction_list_B=None,\
    set_substrate_ini=False,substrates=None,\
    set_A_biomass_ini=None,set_B_biomass_ini=None,biomass_value_A=None,biomass_value_B=None,biomass_id_A=None,biomass_id_B=None,\
    set_bound=False,set_stoi_matrix=False,\
    set_part_enzyme_constraint=False,set_obj_single_E_value=False,E_total_A=None,E_total_B=None,\
    set_obj_value=False,obj_name=None,obj_target=None,\
    set_target_ini=False,target_name=None,target_value=None,\
    set_obj_V_value=False,set_obj_E_value=False,special_constraint=False,x=None,y=None):
    

    Concretemodel = ConcreteModel()
    Concretemodel.reaction = pyo.Var(reaction_list,  within=NonNegativeReals)
    Concretemodel.z = pyo.Var(reaction_list,  within=pyo.Binary)
    Concretemodel.x = pyo.Var()
    Concretemodel.y = pyo.Var()

    #Set upper and lower bounds of metabolite concentration
    # if set_metabolite:
    #     def set_metabolite(m,i):
    #         return  inequality(metabolites_lnC.loc[i,'lnClb'], m.metabolite[i], metabolites_lnC.loc[i,'lnCub'])
    #     Concretemodel.set_metabolite= Constraint(metabolite_list,rule=set_metabolite)     

    #Minimizing the flux sum of pathway (pFBA)
    if set_obj_V_value:             
        def set_obj_V_value(m):
            return sum(m.reaction[j] for j in reaction_list)
        Concretemodel.obj = Objective(rule=set_obj_V_value, sense=minimize)  
    
    #Set the maximum flux as the object function
    if set_obj_value:   
        def set_obj_value(m):
            return m.reaction[obj_name]#+m.reaction['A_'+biomass_id_A]+m.reaction['B_'+biomass_id_B]
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=maximize)
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_value, sense=minimize)

    #Set the minimum enzyme cost of a pathway as the object function
    if set_obj_E_value:             
        def set_obj_E_value(m):
            return sum(m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index)
        Concretemodel.obj = Objective(rule=set_obj_E_value, sense=minimize)  

    #To calculate the variability of enzyme usage of single reaction.
    if set_obj_single_E_value:             
        def set_obj_single_E_value(m):
            return m.reaction[obj_name]/(reaction_kcat_MW.loc[obj_name,'kcat_MW'])
        if obj_target=='maximize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=maximize) 
        elif obj_target=='minimize':
            Concretemodel.obj = Objective(rule=set_obj_single_E_value, sense=minimize)

    #Adding flux balance constraints （FBA）
    if set_stoi_matrix:
        def set_stoi_matrix(m,i):
            return sum(coef_matrix[i,j]*m.reaction[j]  for j in reaction_list if (i,j) in coef_matrix.keys() )==0
        Concretemodel.set_stoi_matrix = Constraint( metabolite_list,rule=set_stoi_matrix)

    #Adding the upper and lower bound constraints of reaction flux
    if set_bound:
        def set_bound(m,j):
            return inequality(lb_list[j],m.reaction[j],ub_list[j])
        Concretemodel.set_bound = Constraint(reaction_list,rule=set_bound) 
    
     #Set the upper bound for substrate input reaction flux
    if set_substrate_ini :
        def set_substrates_ini(m,j): 
            return m.reaction[j]<= substrates[j]
        Concretemodel.set_substrates_ini = Constraint(substrates.keys(),rule=set_substrates_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_A_biomass_ini:
        def set_A_biomass_ini(m): 
            return m.reaction['A_'+biomass_id_A] >=biomass_value_A
        Concretemodel.set_A_biomass_ini = Constraint(rule=set_A_biomass_ini)  
        
    #Set the lower bound for biomass synthesis reaction flux
    if set_B_biomass_ini:
        def set_B_biomass_ini(m): 
            return m.reaction['B_'+biomass_id_B] >=biomass_value_B
        Concretemodel.set_B_biomass_ini = Constraint(rule=set_B_biomass_ini)  
    
    #Set the lower bound for target synthesis reaction flux
    if set_target_ini:
        def set_target_ini(m): 
            return m.reaction[target_name] >=target_value
        Concretemodel.set_target_ini = Constraint(rule=set_target_ini)  
    
    #Adding Part A enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_A(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_A)<= E_total_A
        Concretemodel.set_enzyme_constraint_A = Constraint(rule=set_enzyme_constraint_A)

    #Adding Part B enzymamic constraints
    if set_part_enzyme_constraint:
        def set_enzyme_constraint_B(m):
            return sum( m.reaction[j]/(reaction_kcat_MW.loc[j,'kcat_MW']) for j in reaction_kcat_MW.index if j in reaction_list_B)<= E_total_B
        Concretemodel.set_enzyme_constraint_B = Constraint(rule=set_enzyme_constraint_B)
    
    if special_constraint:
        def special_constraint(m):
            return m.reaction['A_pdx']==m.reaction['A_LYSDC_num1']+m.reaction['A_LYSDC_num2']
        Concretemodel.special_constraint = Constraint(rule=special_constraint)
        # def special_constraint2(m):
        #     return m.reaction['A_'+biomass_id_A]*x+m.reaction['B_'+biomass_id_B]*y>=0.46
        # Concretemodel.special_constraint2 = Constraint(rule=special_constraint2)
    return Concretemodel

In [637]:
def ECM_FBA_diff(Concretemodel_Need_Data,substrates,x,y,bio_a,bio_b,obj_name,obj_target='maximize',set_target_ini=False,target_name=None,target_value=None,special_constraint=False):#基于pyomo的FBA分析方法（glc）
    E_total_A=0.227
    E_total_B=0.129
    sub_a='EX_glc__D_e_reverse'
    sub_b='EX_xyl__D_e_reverse'
    ratio=4
    reaction_list_A=deepcopy(Concretemodel_Need_Data['reaction_list_A'])
    reaction_list_B=deepcopy(Concretemodel_Need_Data['reaction_list_B'])
    reaction_list=deepcopy(Concretemodel_Need_Data['reaction_list'])
    metabolite_list=deepcopy(Concretemodel_Need_Data['metabolite_list'])
    coef_matrix=deepcopy(Concretemodel_Need_Data['coef_matrix'])
    for i in metabolite_list:
        for j in reaction_list:
                if (i,j) in coef_matrix.keys():
                    if j.startswith('A_') and i.startswith('A_') and '_con_' in j:
                        coef_matrix[i,j]=(1/x)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                                # print ('A:'+str(coef_matrix[i,j]))
                    if j.startswith('B_') and i.startswith('B_') and '_con_' in j:
                        coef_matrix[i,j]=(1/y)*coef_matrix[i,j]
                        # if 'akg_e_con_' in j:
                #                 print ('B:'+str(coef_matrix[i,j]))
                #     print ('ij:'+str(coef_matrix[i,j]))
    reaction_kcat_MW=deepcopy(Concretemodel_Need_Data['reaction_kcat_MW'])
    lb_list=Concretemodel_Need_Data['lb_list']
    ub_list=Concretemodel_Need_Data['ub_list']
    biomass_id_a='BIOMASS_Ec_iML1515_core_75p37M'
    biomass_id_b='CG_biomass_cgl_ATCC13032'    
    #pass_value={'A_tyr__L_e_con_1':tyr,'B_phpyr_e_con_2_reverse':php}
    EcoECM_FBA=Template_Concretemodel_double_diff(reaction_list=reaction_list,metabolite_list=metabolite_list,coef_matrix=coef_matrix,\
        reaction_kcat_MW=reaction_kcat_MW,lb_list=lb_list,ub_list=ub_list,reaction_list_A=reaction_list_A,reaction_list_B=reaction_list_B,\
        set_substrate_ini=True,substrates=substrates,\
        set_A_biomass_ini=True,set_B_biomass_ini=True,biomass_value_A=bio_a,biomass_value_B=bio_b,biomass_id_A=biomass_id_a,biomass_id_B=biomass_id_b,\
        set_bound=True,set_stoi_matrix=True,set_obj_single_E_value=False,\
        set_part_enzyme_constraint=True,E_total_A=E_total_A,E_total_B=E_total_B,\
        set_obj_value=True,obj_name=obj_name,obj_target='maximize',\
        set_target_ini=set_target_ini,target_name=target_name,target_value=target_value,\
        set_obj_V_value=False,set_obj_E_value=False,special_constraint=special_constraint,x=x,y=y)
    opt_ecm_FBA=Model_Solve(EcoECM_FBA,'gurobi')
    #opt_ecm_pFBA.obj()
    return opt_ecm_FBA

# continue

In [793]:
substrates={'EX_glyc_e_reverse':400,'EX_lys__L_e_reverse':0}

In [764]:
Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num1','kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num1','kcat_MW']*2

In [765]:
Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num2','kcat_MW']=Concretemodel_Need_Data['reaction_kcat_MW'].loc['A_LYSDC_num2','kcat_MW']*2

Cgl:BL=3:1

In [832]:
result=ECM_FBA_diff(Concretemodel_Need_Data,substrates=substrates,x=1/4,y=3/4,bio_a=0.1,bio_b=0.23,obj_name='EX_15dap_e',obj_target='maximize',special_constraint=True)

In [833]:
result.obj()

1.2790436623513766

In [834]:
result.reaction['EX_15dap_e'].value

1.2790436623513766

Cgl:BL=2:1

In [835]:
result=ECM_FBA_diff(Concretemodel_Need_Data,substrates=substrates,x=1/3,y=2/3,bio_a=0.1,bio_b=0.23,obj_name='EX_15dap_e',obj_target='maximize',special_constraint=True)

In [836]:
result.obj()

1.7053915498018395

Cgl:BL=1:1

In [837]:
result=ECM_FBA_diff(Concretemodel_Need_Data,substrates=substrates,x=1/2,y=1/2,bio_a=0.26,bio_b=0.23,obj_name='EX_15dap_e',obj_target='maximize',special_constraint=True)

In [838]:
result.obj()

1.6386459522457542

Cgl:BL=1:2

In [839]:
result=ECM_FBA_diff(Concretemodel_Need_Data,substrates=substrates,x=2/3,y=1/3,bio_a=0.26,bio_b=0.1,obj_name='EX_15dap_e',obj_target='maximize',special_constraint=True)

In [840]:
result.obj()

2.184861269660973

Cgl:BL=1:3

In [841]:
result=ECM_FBA_diff(Concretemodel_Need_Data,substrates=substrates,x=3/4,y=1/4,bio_a=0.26,bio_b=0.1,obj_name='EX_15dap_e',obj_target='maximize',special_constraint=True)

In [842]:
result.obj()

2.3634813443017935

In [843]:
result.reaction['A_pdx'].value

3.1513084590690577

In [844]:
result.reaction['A_LYSDC_num1'].value

3.1513084590690577

In [845]:
result.reaction['A_LYSDC_num2'].value

0.0